In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import matplotlib.image as Image
import cv2
import pandas as pd
import math
from sklearn.cluster import KMeans

In [5]:
df_origin = pd.read_csv('data_gen/creditcard.csv')
df_origin.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
df_origin.iloc[:, -1].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [7]:
import sys
sys.path.append('/Users/luoyonggui/PycharmProjects/mayiutils_n1/mayiutils/data_prepare')

In [8]:
from data_explore import DataExplore as de

In [9]:
de.describe(df_origin)

num of records: 284807, num of columns: 31


,Data Type,Unique Values,% Missing,Mode,Count Mode,% Mode,mean,std,min,25%,50%,75%,max
Time,float64,124592,0.0,163152.000000,36,0.0126401,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,float64,275663,0.0,1.245674,77,0.0270359,3.919560e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,float64,275663,0.0,-0.326668,77,0.0270359,5.688174e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,float64,275663,0.0,-2.752041,77,0.0270359,-8.769071e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,float64,275663,0.0,-0.842316,77,0.0270359,2.782312e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,float64,275663,0.0,-0.562777,77,0.0270359,-1.552563e-15,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,float64,275663,0.0,-1.011073,77,0.0270359,2.010663e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,float64,275663,0.0,-0.432126,77,0.0270359,-1.694249e-15,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,float64,275663,0.0,-0.160211,77,0.0270359,-1.927028e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,float64,275663,0.0,0.170362,77,0.0270359,-3.137024e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


In [10]:

# df = df_origin.apply(lambda x: abs(x+2)**0.5)
df = df_origin
data = df.values[:, 1:]
test = data[(data[:, -1] == 1)]
data = data[(data[:, -1] == 0)]
data = data[:, :-1]
test = test[:, :-1]

In [11]:
data_max = np.max(data, axis=0)
data_min = np.min(data, axis=0) + 1e-6

In [12]:
# 数据归一化
data = (data - data_min) / (data_max - data_min)
test = (test - data_min) / (data_max - data_min)

In [32]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], data.shape[1]))
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [14]:
inputs = keras.Input(shape=(data.shape[1],))
out = keras.layers.Dense(10, activation='relu')(inputs)

z_mean_layer = keras.layers.Dense(data.shape[1])
z_log_var_layer = keras.layers.Dense(data.shape[1])

z_mean = z_mean_layer(out)
z_log_var = z_log_var_layer(out)

z = keras.layers.Lambda(sampling)([z_mean, z_log_var])

mean_decoded = keras.layers.Dense(data.shape[1], activation='relu')(z)

vae = keras.Model(inputs, mean_decoded)

generate = keras.Model(inputs, [z_mean, z_log_var])

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
generate.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 29)           0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 10)           300         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 29)           319         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 29)           319         dense[0][0]                      
Total params: 938
Trainable params: 938
Non-trainable params: 0
_____________________________________________

In [16]:
xent_loss = data.shape[1] * keras.losses.MSE(K.reshape(inputs, (-1, data.shape[1])), mean_decoded)
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

vae.add_loss(vae_loss)

In [17]:
vae.compile('adam')

In [40]:
vae.fit(data, shuffle=True, epochs=10, batch_size=512)

Epoch 1/10
284315/284315 [==============================] - 1s 4us/sample - loss: 0.0567
Epoch 2/10
284315/284315 [==============================] - 1s 4us/sample - loss: 0.0565
Epoch 3/10
284315/284315 [==============================] - 1s 3us/sample - loss: 0.0564
Epoch 4/10
284315/284315 [==============================] - 1s 3us/sample - loss: 0.0562 0s -
Epoch 5/10
284315/284315 [==============================] - 1s 3us/sample - loss: 0.0561
Epoch 6/10
284315/284315 [==============================] - 1s 3us/sample - loss: 0.0560
Epoch 7/10
284315/284315 [==============================] - 1s 4us/sample - loss: 0.0559
Epoch 8/10
284315/284315 [==============================] - 1s 4us/sample - loss: 0.0558
Epoch 9/10
284315/284315 [==============================] - 1s 4us/sample - loss: 0.0558
Epoch 10/10
284315/284315 [==============================] - 1s 4us/sample - loss: 0.0557


# 生成模型

In [90]:
def caculate(x, x_hat):
#     result = 1 - np.mean(np.square((np.reshape(x, (data.shape[1],)) - x_hat)**2))
#     result = np.dot(x.flatten(), x_hat.flatten()) / (np.linalg.norm(x.flatten())*np.linalg.norm(x_hat.flatten()))
    p = []
    for i in range(len(x)):
        p.append(x[i].dot(x_hat[i])/(np.linalg.norm(x[i])*np.linalg.norm(x_hat[i])))
#     mean_by_z, _ = generate.predict(x)
#     mean_by_x, _ = generate.predict(x_hat)
#     p = 1 - np.mean((mean_by_z - mean_by_x)**2, axis=1)**0.5
#     print(p)
    return np.array(p)


# 正样本

In [42]:
origin = data[2:3]
origin

array([[0.93521702, 0.77905459, 0.86814082, 0.26876547, 0.76232878,
        0.28112211, 0.2136886 , 0.79916907, 0.21822788, 0.38841782,
        0.36635794, 0.66154134, 0.50385419, 0.63025122, 0.50773584,
        0.26340865, 0.69096971, 0.50398123, 0.38668308, 0.58585504,
        0.61063999, 0.54602981, 0.67893916, 0.28935377, 0.55951517,
        0.40272708, 0.41548926, 0.3119113 , 0.01473892]])

In [43]:
result = vae.predict(origin)[0]
result

array([0.95700455, 0.79844517, 0.83551496, 0.25606352, 0.7645944 ,
       0.25666487, 0.20648107, 0.8004253 , 0.29565454, 0.39258087,
       0.32134306, 0.65553325, 0.4421999 , 0.6386863 , 0.3378005 ,
       0.3771497 , 0.6488419 , 0.5285195 , 0.5597626 , 0.57969683,
       0.60710305, 0.5085343 , 0.6636669 , 0.3612535 , 0.58134   ,
       0.43882754, 0.4127423 , 0.3128984 , 0.        ], dtype=float32)

## 有正常样本生成的样本也是和原样本的余弦相似度很高

In [44]:
result = np.dot(origin.flatten(), result.flatten()) / (np.linalg.norm(origin.flatten())*np.linalg.norm(result.flatten()))
result

0.9943295940518977

# 负样本

In [45]:
origin = test[2:3]
result = vae.predict(origin)[0]
result = np.dot(origin.flatten(), result.flatten()) / (np.linalg.norm(origin.flatten())*np.linalg.norm(result.flatten()))
print(result)

0.9870190949627414


In [91]:
origin = data[2:3]
result = vae.predict(origin)
caculate(origin, result)

array([0.99402755])

In [92]:
origin = test[2:3]
result = vae.predict(origin)
caculate(origin, result)

array([0.98716883])

In [125]:
epslion = 0.974

# 统计结果

In [126]:
pre = vae.predict(data)

result = caculate(data, pre)
result = result < epslion

pre = vae.predict(test)
anomaly_result = caculate(test, pre)
anomaly_result = anomaly_result < epslion


In [127]:
from sklearn.metrics import classification_report

In [128]:
# epslion = 0.974
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.46      0.49      0.48       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.73      0.75      0.74    284807
weighted avg       1.00      1.00      1.00    284807



In [124]:
# epslion = 0.97
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.53      0.45      0.48       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.76      0.72      0.74    284807
weighted avg       1.00      1.00      1.00    284807



In [120]:
# epslion = 0.98
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.35      0.59      0.44       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.68      0.79      0.72    284807
weighted avg       1.00      1.00      1.00    284807



In [116]:
# epslion = 0.982
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.32      0.61      0.42       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.66      0.80      0.71    284807
weighted avg       1.00      1.00      1.00    284807



In [112]:
# epslion = 0.984
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.28      0.62      0.38       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.64      0.81      0.69    284807
weighted avg       1.00      1.00      1.00    284807



In [108]:
# epslion = 0.986
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.22      0.66      0.33       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.61      0.83      0.67    284807
weighted avg       1.00      1.00      1.00    284807



In [104]:
# epslion = 0.99
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    284315
           1       0.11      0.79      0.20       492

   micro avg       0.99      0.99      0.99    284807
   macro avg       0.56      0.89      0.60    284807
weighted avg       1.00      0.99      0.99    284807



In [100]:
# epslion = 0.989
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    284315
           1       0.14      0.77      0.24       492

   micro avg       0.99      0.99      0.99    284807
   macro avg       0.57      0.88      0.62    284807
weighted avg       1.00      0.99      0.99    284807



In [96]:
# epslion = 0.988
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    284315
           1       0.17      0.73      0.28       492

   micro avg       0.99      0.99      0.99    284807
   macro avg       0.58      0.86      0.64    284807
weighted avg       1.00      0.99      1.00    284807



In [76]:
# epslion = 0.994
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99    284315
           1       0.05      0.82      0.09       492

   micro avg       0.97      0.97      0.97    284807
   macro avg       0.52      0.89      0.54    284807
weighted avg       1.00      0.97      0.98    284807



In [73]:
# epslion = 0.991
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    284315
           1       0.16      0.68      0.26       492

   micro avg       0.99      0.99      0.99    284807
   macro avg       0.58      0.84      0.63    284807
weighted avg       1.00      0.99      1.00    284807



In [66]:
# epslion = 0.993
print(classification_report([0]*len(result)+[1]*len(anomaly_result), list(map(int, list(result)+list(anomaly_result)))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.23      0.62      0.34       492

   micro avg       1.00      1.00      1.00    284807
   macro avg       0.62      0.81      0.67    284807
weighted avg       1.00      1.00      1.00    284807

